In [38]:
import numpy as np
import pandas as pd
import ast
import nltk

In [39]:
df_train=pd.read_csv('train.csv')
df_train

,id,product_description,price
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر\r\nپورت ی...,1080000
1,1,{'مشخصات فنی': 'CABLE LENTH:\r\n10feet \\/3M\r...,990000
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB\r\nطول کابل:...,1680000
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000
...,...,...,...
68835,68835,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",2530000
68836,68836,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",22000000
68837,68837,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",4000000
68838,68838,"{'کابل همراه': ['لایتنینگ'], 'ولتاژ خروجی': ['...",4300000


In [40]:
import nltk
import hazm
stopwords=[]
nltk.download('stopwords')
nltk_stopwords=nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
with open ('stopwords.txt',encoding='utf8') as stopwords_file:
    st=stopwords_file.readlines()
stopwords1=[line.replace('\n','')for line in st]
stopwords1.extend(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aghil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
stemmer = SnowballStemmer("english")
stemmer1=hazm.Stemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aghil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
dataset=pd.DataFrame(columns=('product_description','price'))
for index,row in df_train.iterrows():
    title_body=row['product_description']
    title_body_tok=nltk.word_tokenize(title_body)
    title_body_tok_filtered=[w for w in title_body_tok if not w in stopwords1]
    title_body_tok_filtered_stem= [stemmer.stem(w) for w in title_body_tok_filtered]
    title_body_tok_filtered_stem= [stemmer1.stem(w) for w in title_body_tok_filtered]
    dataset.loc[index]={
        'product_description': ' '.join(title_body_tok_filtered_stem),
        'price': row['price']
}

In [43]:
dataset

,product_description,price
0,'مشخص فن 'متراژ 150 سانت متر\r\nپور 'دسته 'کاب...,1080000
1,'مشخص فن 'CABLE LENTH \r\n10feet \\/3M\r\nPC S...,990000
2,'مشخص فن 'نوع کابل KVM USB\r\nطول کابل 1.5 'د...,1680000
3,'دسته 'کابل کمبو 'برند 'متفرقه,4500000
4,'مشخص فن 'کیبورد 6pin Mini-DIN Male Purple \r\...,700000
...,...,...
68835,'دسته 'شارژر تبل موبایل 'برند 'موکسو,2530000
68836,'دسته 'شارژر تبل موبایل 'برند 'ال ج,22000000
68837,'دسته 'شارژر تبل موبایل 'برند 'مورف,4000000
68838,'کابل همراه 'لایتنینگ 'ولتاژ خروج '5.0 ول 'شد ...,4300000


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
vector=TfidfVectorizer()
vector.fit(dataset['product_description'])

TfidfVectorizer()

In [46]:
x=vector.transform(dataset['product_description'])

In [47]:
x

<68840x57897 sparse matrix of type '<class 'numpy.float64'>'
	with 2697918 stored elements in Compressed Sparse Row format>

In [48]:
y=dataset['price']/dataset['price'].max()

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X_train , X_test,Y_train , Y_test=train_test_split(x,y,test_size=0.2)

In [51]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [56]:
params={
        
        
       }

In [57]:
model=RandomForestRegressor()

In [58]:
clf=GridSearchCV(model,param_grid=params,cv=3,verbose=100)

In [59]:
clf.fit(x,y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3; 1/1] START ............................................................
[CV 1/3; 1/1] END ............................., score=-0.069 total time=94.1min
[CV 2/3; 1/1] START ............................................................
[CV 2/3; 1/1] END ............................, score=-0.379 total time=112.8min
[CV 3/3; 1/1] START ............................................................
[CV 3/3; 1/1] END ............................, score=-0.136 total time=116.5min


KeyboardInterrupt: 

In [ ]:
clf.score(X_test,Y_test)

In [238]:
model.score(X_train,Y_train)

0.932767013458741

In [212]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #plt.ylim([12000, 60000])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)

In [213]:
from pandas.core.reshape.melt import lreshape
def scheduler(epoch, lr):
    if epoch==320:
         return lr +0.0004
    if epoch==500:
         return lr *0.1
    if epoch==600:
         return lr *0.1
    else:
         return lr

In [214]:
import tensorflow as tf

In [219]:
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Input(57897))
model2.add(tf.keras.layers.Dense(1111,activation='relu',kernel_regularizer='l1'))
model2.add(tf.keras.layers.Dense(888,activation='elu',kernel_regularizer='l1'))
model2.add(tf.keras.layers.Dropout(0.25))
model2.add(tf.keras.layers.Dense(770,activation='relu',kernel_regularizer='l1',kernel_initializer=tf.keras.initializers.he_normal))
model2.add(tf.keras.layers.Dropout(0.25))
model2.add(tf.keras.layers.Dense(1,activation='elu',kernel_regularizer='l1'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1111)              64324678  
                                                                 
 dense_5 (Dense)             (None, 888)               987456    
                                                                 
 dropout_2 (Dropout)         (None, 888)               0         
                                                                 
 dense_6 (Dense)             (None, 770)               684530    
                                                                 
 dropout_3 (Dropout)         (None, 770)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 771       
                                                                 
Total params: 65,997,435
Trainable params: 65,997,435


In [222]:
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss= tf.keras.losses.MeanSquaredError(),
    metrics=tf.keras.losses.MeanAbsoluteError()
)

In [223]:
h=model2.fit(x=X_train,y=Y_train,batch_size=1,epochs=1500,validation_split=0.2)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'scipy.sparse._csr.csr_matrix'>]

In [178]:
df_test=pd.read_csv('test.csv')

In [181]:
dataset1=pd.DataFrame(columns=('product_description',))
for index,row in df_test.iterrows():
    title_body=row['product_description']
    title_body_tok=nltk.word_tokenize(title_body)
    title_body_tok_filtered=[w for w in title_body_tok if not w in stopwords1]
    title_body_tok_filtered_stem= [stemmer.stem(w) for w in title_body_tok_filtered]
    title_body_tok_filtered_stem= [stemmer1.stem(w) for w in title_body_tok_filtered]
    dataset1.loc[index]={
        'product_description': ' '.join(title_body_tok_filtered_stem),
}

In [182]:
xt=vector.transform(dataset1['product_description'])

In [187]:
xxx={}

In [188]:
xxx['id']=np.array(df_test['id'])
xxx['price']=model.predict(xt)*dataset['price'].max()

In [189]:
xxx

{'id': array([    0,     1,     2, ..., 34259, 34260, 34261], dtype=int64),
 'price': array([ 863234.95607107,  863234.95607107, 6128599.73872779, ...,
        2095583.17113192, -434544.01401581, 3821573.46933496])}

In [193]:
pd.DataFrame(xxx['price'],index=xxx['id']).to_csv('output.csv')